Correr una regresión lineal multiple resulta sencillo, puesto que el módulo de <span style="background-color: #D3D3D3; padding: 2px;"> statsmodels</span> tiene implementado una gran cantidad de ssubmódulos, clases y funciones relacionados con la Estadística.
Consideremos el siguiente modelo de regresión poblacional:

$$y_i = \beta_0 + \beta_1 x_{1,i} + \beta_2 x_{2,i} + ... + \beta_k x_{k,i} + u_i$$

Un método un importante que nos interesá sera el método <span style="background-color: #D3D3D3; padding: 2px;"> summary()</span>, ya que este método contendra muchos resultados útiles, tales como las estimaciones de lo parámetros, los errores estandar de la edtiamción, los errores estandar de la regresión, el $R^2$, entre otros que no abordamos aún.

Para ejemplificar la estimación del modelo de regresión lineal multiple, se replicara, al igual que el cuaderno 2, los resultados del libro base que tenemos.

Sin embargo, antes de continuar es conveniente definir el modelo de Mínimos Cuadrados Ordinarios en su versión matricial.
En forma matricial, almacenamos los regresores en una matriz $X$ de tamaño $ n \times k$ (donde k es el número de parámetros a estimar) que tiene una columna para cada regresor más una columna de unos para la constante. Los valores de muestra de la variable dependiente se almacenan en un vector $Y $ de tamaño $n \times 1$.

La Suma de Cuadrados Residuales se define de la siguiente manera:
$$S_T(\beta) = \left(Y - X\beta \right)' \left(Y - X \beta\right)$$
Si minimizamos dicha función podemos llegar a la siguiente ecuación:
$$\hat{\beta} = (X'X)^{-1}(X'Y)$$

Una nota importante, además de otras que no la cubrire, es que para poder estimar $\beta$, $X'X$ debe ser de rango completo. Es decir no debe haber colinealidad perfecta entre los regresores.

El vector de residuos se puede calcular como:
$$\hat{u} = Y - X \hat{\beta}$$


La fórmula para la varianza estimada del término de error es:
$$\hat{\sigma}^2 = \frac{1}{n-k} \hat{u}' \hat{u}$$

El error estándar de la regresión es su raiz cuadrada $\sigma = \sqrt{\sigma^2}$. La matriz de varianza y covarianza del estimador es la siguiente:
$$\text{Var}(\hat{\beta}) = \hat{\sigma}^2 (X'X)^{-1}$$
Donde, si sacamos la raiz cuadrada a la diagonal principal podremos obtener el error estandar de los estimadores de los parámetros poblacionales.

# Data: Determinants of college GPA

Para este ejemplo tomaremos la base de datos de <font color = red> GPA1 </font>, que relaciona el promedio general de calificaciones de la universidad (colGPA) el promedio general de calificaciones en el bachillerato (hsGPA)
y la puntuación en el examen de admisión a la universidad (ACT) para una muestra de 141 estudiantes.
El modelo de regresión poblacional propuesto es el siguiente:
$$\text{colGPA} = \beta_0 + \beta_1 \text{hsGPA} + \beta_3 {ACT} + u$$

In [1]:
import wooldridge as woo
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import patsy as pt
import statsmodels.formula.api as smf
plt.style.use('ggplot')

In [2]:
gpa1 = woo.dataWoo('GPA1')

In [3]:
gpa1.columns

Index(['age', 'soph', 'junior', 'senior', 'senior5', 'male', 'campus',
       'business', 'engineer', 'colGPA', 'hsGPA', 'ACT', 'job19', 'job20',
       'drive', 'bike', 'walk', 'voluntr', 'PC', 'greek', 'car', 'siblings',
       'bgfriend', 'clubs', 'skipped', 'alcohol', 'gradMI', 'fathcoll',
       'mothcoll'],
      dtype='object')

In [4]:
# Progamamos lo mencionado anteriormente:
# Determinar el tamaño de la muestra y el Nro de regresores
n = len(gpa1) 
k = 3 #No olivar que X contiene una columna de constantes

In [5]:
# Estraemos Y
Y = gpa1['colGPA']
# Extraemos X y adicionamos una columna de unos
X = pd.DataFrame({'Constants': 1,
                 'hsGPA': gpa1['hsGPA'],
                 'ACT': gpa1['ACT']})
# Forma alternativa con patsy:
# y2, X2 = pt.dmatrices(’colGPA ~ hsGPA + ACT’, data=gpa1, return_type=’dataframe’)

In [6]:
# Parámetros estimados
X = np.array(X)
Y = np.array(Y).reshape(n, 1) # lo convertimos en un vector fila
b = np.dot(np.linalg.inv(np.dot(X.T, X)), np.dot(X.T, Y))
# otra forma interesante de multiplicar matrices:
# b1 = np.linalg.inv(X.T @ X) @ X.T @ Y 
print(f'b: \n{b}\n')

# residuales, estimamos la varianza de u y su error estandar (SER):
u_hat = Y - np.dot(X, b)
sigsq_hat = (np.dot(u_hat.T, u_hat)) / (n - k)
SER = np.sqrt(sigsq_hat)
print(f'SER: {SER}\n')

# estimamos la varianza de los parámetros y su error estandar
Vbeta_hat = sigsq_hat * np.linalg.inv(np.dot(X.T, X))
se = np.sqrt(np.diagonal(Vbeta_hat))
print(f'se: {se}\n')

b: 
[[1.28632777]
 [0.45345589]
 [0.00942601]]

SER: [[0.34031576]]

se: [0.34082212 0.09581292 0.01077719]



Podemos comprobar estos resultados obtendidos usando <span style="background-color: #D3D3D3; padding: 2px;"> statsmodels</span>

In [7]:
# Calculamos la función de regresión MCO:
reg = smf.ols(formula='colGPA ~ hsGPA + ACT', data=gpa1)
results = reg.fit()
print(f'Usar summary(): \n{results.summary()}\n')

Usar summary(): 
                            OLS Regression Results                            
Dep. Variable:                 colGPA   R-squared:                       0.176
Model:                            OLS   Adj. R-squared:                  0.164
Method:                 Least Squares   F-statistic:                     14.78
Date:                Sat, 24 Feb 2024   Prob (F-statistic):           1.53e-06
Time:                        04:51:21   Log-Likelihood:                -46.573
No. Observations:                 141   AIC:                             99.15
Df Residuals:                     138   BIC:                             108.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.2863      0.341   

Omitir regresores relevantes causa sesgo si estamos interesados en estimación de efectos parciales. TAmbién como se vera más adelante, omitir variables relevantes causa que nuestros estimadores no sean consistentes.

In [8]:
b = results.params
# relación entre regresores:
reg_delta = smf.ols(formula='hsGPA ~ ACT', data=gpa1)
results_delta = reg_delta.fit()
delta_tilde = results_delta.params
print(f'delta_tilde: \n{delta_tilde}\n')
# Fórmula de variables omitidas para b1_tilde:
b1_tilde = b['ACT'] + b['hsGPA'] * delta_tilde['ACT']
print(f'b1_tilde: \n{b1_tilde}\n')
# regresión real con hsGPA omitida:
reg_om = smf.ols(formula='colGPA ~ ACT', data=gpa1)
results_om = reg_om.fit()
b_om = results_om.params
print(f'b_om: \n{b_om}\n')

delta_tilde: 
Intercept    2.462537
ACT          0.038897
dtype: float64

b1_tilde: 
0.027063973943178526

b_om: 
Intercept    2.402979
ACT          0.027064
dtype: float64



# Data: equation for hourly wage

Para este segundo ejemplo se usara la base de datos <font color = red> WAGE1 </font>. Donde relacionamos los años de aducación (educ), años de experiencia en el mercado laboral (exper) y los años de antiguedad en el empleo actual (tenure) para explicar el salario (wage). Para una muestra de 526 trabajadores.
Proponemos el siguiente modelo de regresión poblacional:

$$\text{wage} = \beta_0 + \beta_1 \text{educ} + \beta_2 \text{exper} + \beta_3 \text{tenure} + u$$

In [9]:
# Base de datos
wage1 = woo.dataWoo('WAGE1')

In [10]:
wage1.columns

Index(['wage', 'educ', 'exper', 'tenure', 'nonwhite', 'female', 'married',
       'numdep', 'smsa', 'northcen', 'south', 'west', 'construc', 'ndurman',
       'trcommpu', 'trade', 'services', 'profserv', 'profocc', 'clerocc',
       'servocc', 'lwage', 'expersq', 'tenursq'],
      dtype='object')

In [11]:
# Regresionamos el modelo MCO
# Usar un modelo semi log, donde transformamos la variable wage en log
reg = smf.ols(formula='np.log(wage) ~ educ + exper + tenure', data=wage1)
results = reg.fit()
print(f'Resultados: \n{results.summary()}\n')

Resultados: 
                            OLS Regression Results                            
Dep. Variable:           np.log(wage)   R-squared:                       0.316
Model:                            OLS   Adj. R-squared:                  0.312
Method:                 Least Squares   F-statistic:                     80.39
Date:                Sat, 24 Feb 2024   Prob (F-statistic):           9.13e-43
Time:                        04:51:21   Log-Likelihood:                -313.55
No. Observations:                 526   AIC:                             635.1
Df Residuals:                     522   BIC:                             652.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2844      0.104      2